In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.preprocessing import LabelEncoder
import pickle


In [2]:
base_model = ResNet50(weights="imagenet", include_top=False, pooling="avg")

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [3]:
# Function to Extract Features
def extract_features(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize for ResNet50
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    features = base_model.predict(img)
    return features.flatten()  # Convert to 1D

In [4]:
# Directory Containing Face Images
dataset_path = "D:/SREC/Data"  # Folder with subfolders for each person
feature_list = []
labels = []


In [5]:
# Process each image

for person in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person)
    if os.path.isdir(person_folder):
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            feature_vector = extract_features(img_path)
            feature_list.append(feature_vector)
            labels.append(person)  # Folder name is the class label

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [6]:
# Convert to NumPy Array
features = np.array(feature_list)
labels = np.array(labels)

In [7]:

# Encode Labels
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)


In [8]:
# Save Features and Labels
with open("face_features.pkl", "wb") as f:
    pickle.dump({"features": features, "labels": encoded_labels, "label_encoder": le}, f)

print("Feature extraction completed and saved!")

Feature extraction completed and saved!


In [9]:
import pickle
from sklearn.neighbors import KNeighborsClassifier

# Load Stored Features
with open("face_features.pkl", "rb") as f:
    data = pickle.load(f)

features = data["features"]
labels = data["labels"]
label_encoder = data["label_encoder"]

# Train a KNN Classifier
knn = KNeighborsClassifier(n_neighbors=3, metric="euclidean")
knn.fit(features, labels)

# Function to Classify a New Image
def classify_face(image_path):
    feature_vector = extract_features(image_path)
    feature_vector = feature_vector.reshape(1, -1)
    
    # Predict Class
    prediction = knn.predict(feature_vector)
    person_name = label_encoder.inverse_transform(prediction)[0]
    
    return person_name

In [11]:
# Test with a New Image
test_image_path = "D:/SREC/Data/Vijay/71Zg6RRQzsL.jpg"  # Provide test image path
predicted_person = classify_face(test_image_path)
print(f"Predicted Person: {predicted_person}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step
Predicted Person: Vijay
